## Fluid Properties
Using carbatpy, the default is evaluating fluid properties with REFPROP, but along initializing a fluid you can also set props="TREND". You should have installed the package you want to use, they are not part of carbatpy!

In [9]:
import carbatpy as cb

Select a fluid, calculate a state using the default REFPROP and print it out:

In [10]:
FLUID = "Propane * Pentane"
    
comp = [.5, .5]
myFluid = cb.init_fluid(FLUID, comp)

quality = 0.5       # is interpreted by carbatpy as mol/mol to calculate state
# but given back in string as kg/kg
st0 = myFluid.set_state([300., quality], "TQ")
myFluid.print_state()


Propane * Pentane, composition: [0.5, 0.5]
Quality: 0.500
Temperature    : 300.0
Pressure    : 252349.40139512054
spec_Enthalpy    : 269514.7017616625
spec_Volume    : 0.09179168154889315
spec_Entropy    : 1152.0098672722488
quality_mass    : 0.5
spec_internal_Energy    : 246351.13772241608
x    : [0.18381679 0.81618321]
y    : [0.74224573 0.25775427]
z    : [0.5 0.5]


## Properties and Units
The properties can be accessed from a dictionary. **Units are SI** (mass base: K, kg. Pa, m), only composition is always in **mole fractions**.

In [11]:
myFluid.val_dict

{'Temperature': 300.0,
 'Pressure': 252349.40139512054,
 'spec_Enthalpy': 269514.7017616625,
 'spec_Volume': 0.09179168154889315,
 'spec_Entropy': 1152.0098672722488,
 'quality_mass': 0.5,
 'spec_internal_Energy': 246351.13772241608,
 'x': array([0.18381679, 0.81618321]),
 'y': array([0.74224573, 0.25775427]),
 'z': array([0.5, 0.5])}

or using the properties (FluidState Class):

In [12]:
myFluid.properties.enthalpy

269514.7017616625

You are interested in transport properties? First check the Default settings:

In [13]:
cb.CB_DEFAULTS

{'Fluid_Defaults': {'PROPS': 'REFPROP',
  'DLL_SELECT': '2dll',
  'UNITS': 'MASS BASE SI',
  'Property_Names': ('Temperature',
   'Pressure',
   'spec_Enthalpy',
   'spec_Volume',
   'spec_Entropy',
   'quality_mass',
   'spec_internal_Energy',
   'viscosity',
   'thermal_conductivity',
   'Prandtl_number',
   'isobaric_heat_capacity',
   'speed_of_sound',
   'molecular_mass'),
  'Property_Names_Short': {'temperature': 'T',
   'pressure': 'p',
   'enthalpy': 'h',
   'sp_volume': 'v',
   'entropy': 's',
   'quality': 'q',
   'int_energy': 'u',
   'viscosity': 'eta',
   'thermal_conductivity': 'k',
   'prandtl': 'Pr',
   'cp': 'cp',
   'speed_of_sound': 'w_s',
   'molecular_mass': 'M'},
  'THERMO_STRING': 'T;P;H;V;S;QMASS;E',
  'TRANS_STRING': 'T;P;H;V;S;QMASS;E;VIS;TCX;PRANDTL;CP;W;M',
  'THERMO_TREND': ['T', 'P', 'H', 'D', 'S', 'QEOS', 'U'],
  'TRANS_TREND': ['T',
   'P',
   'H',
   'D',
   'S',
   'QEOS',
   'U',
   'ETA',
   'TCX',
   'CP',
   'WS'],
  'TREND': {'TREND_INSTALLED': Tr

Then we need the TRANS_STRING the default is the THERMOSTRING:

In [14]:
TS = cb.CB_DEFAULTS['Fluid_Defaults']['TRANS_STRING']
state = myFluid.set_state([300, 1e5], "TP", TS)

In [15]:
myFluid.val_dict

{'Temperature': 300.0,
 'Pressure': 100000.0,
 'spec_Enthalpy': 453021.3769789692,
 'spec_Volume': 0.4170208242631057,
 'spec_Entropy': 1871.0035937100859,
 'quality_mass': -9999990.0,
 'spec_internal_Energy': 411319.29455265857,
 'viscosity': 7.431933119286727e-06,
 'thermal_conductivity': 0.016293724405010007,
 'Prandtl_number': 0.7777062281522813,
 'isobaric_heat_capacity': 1705.0383454997057,
 'speed_of_sound': 211.58293060385915,
 'molecular_mass': 0.0581222,
 'x': None,
 'y': None,
 'z': array('d', [0.5, 0.5])}

## Return Value
By default the returned output (state) is a list (in the same order, as above, but without x,y, z), which may be more convenient to use, but the order of the properties after the internal energy may change in later versions.
by setting the keyword "output" = "FluidState", you can get an instanse of FluidState as return value and you can access the properties by name (here: spec. enthalpy):



In [19]:
my_prop = myFluid.set_state([300, 1e5], "TP",output="FluidState")
print("FluidState:\n",my_prop.state_to_dict(),f"\nEnthalpy: {my_prop.enthalpy}\n")
print(f"\nstate:\n {state}")

FluidState:
 {'Temperature': 300.0, 'Pressure': 100000.0, 'spec_Enthalpy': 453021.3769789692, 'spec_Volume': 0.4170208242631057, 'spec_Entropy': 1871.0035937100859, 'quality_mass': -9999990.0, 'spec_internal_Energy': 411319.29455265857, 'x': None, 'y': None, 'z': array('d', [0.5, 0.5])} 
Enthalpy: 453021.3769789692


state:
 [ 3.00000000e+02  1.00000000e+05  4.53021377e+05  4.17020824e-01
  1.87100359e+03 -9.99999000e+06  4.11319295e+05  7.43193312e-06
  1.62937244e-02  7.77706228e-01  1.70503835e+03  2.11582931e+02
  5.81222000e-02]


## TREND
If you installed it, you can use it. It needs some additional input (see TREND manual). Although we initialize with "TP" (temperature, pressure), you can use it with other combinations also, if they are installed in TREND. The units are as said above.
Transport property evaluation will be implemented soon, but the calculated values differ from those in REFPROP and currently mixtur transport properties cannot be evaluated.
Both cannot handle two-phase fluidtransport properties!

In [ ]:
TREND = cb.CB_DEFAULTS['Fluid_Defaults']['TREND']

if TREND["TREND_INSTALLED"]:
    trend_dll = TREND["TREND_DLL"]
    trend_path = TREND["TREND_PATH"]
    
tr_dict = {"Input": "TP",
               'calctype': "H",
               'fluids': FLUID,
               "moles": comp,
               "eos_ind": [1, 1],
               'mix_ind': 1,
               'path': trend_path,
               'unit': 'specific',
               'dll_path': trend_dll}
my_trend_fluid = cb.init_fluid(FLUID, comp, props="TREND", args= tr_dict)

In [ ]:
state = my_trend_fluid.set_state([300, 1e5], "TP")
my_trend_fluid.val_dict